In [1]:
import os
import csv
import json
import argparse
import collections

In [5]:
def read_org_info(filename):
    orgid_to_countries = {}
    orgid_to_asns = collections.defaultdict(set)
    count = 0
    with open(filename, 'r') as f1:
        line = f1.readline()
        while line:
            data = []
            while line.startswith('#'):
                line = f1.readline()
                if not line:
                    break
            while not line.startswith('#'):
                data.append(line)
                line = f1.readline()
                if not line:
                    break
            reader = csv.reader(data, delimiter='|')
            for row in reader:
                if count == 0:
                    orgid_to_countries[row[0]] = row[3]
                elif count == 1:
                    orgid_to_asns[row[3]].add(row[0])
            count += 1
            
    asn_to_country_caida = {}
    for key in orgid_to_countries.keys():
        for asn in orgid_to_asns[key]:
            asn_to_country_caida[asn] = orgid_to_countries[key]

    return asn_to_country_caida


def read_country_to_continent(filename):
    country_to_continent_d = dict()
    with open(filename, 'r') as f2:
        next(f2)
        reader = csv.reader(f2, delimiter=',')
        country_to_continent_d = { key: value for key, value in reader }
    return country_to_continent_d

In [8]:
results_folder = "/media/gerben/Seagate/Study/hacking_lab/HackingLab_7years/Analysis/results/"
caida_info_file = "/media/gerben/Seagate/Study/hacking_lab/HackingLab_7years/datasets/as_to_organization_info/20191001.as-org2info.txt"
coun_to_cont = "/media/gerben/Seagate/Study/hacking_lab/HackingLab_7years/datasets/country_to_continent/country_to_continent.txt"


asn_to_country_caida_d = read_org_info(caida_info_file)
country_to_continent_d = read_country_to_continent(coun_to_cont)
#print(asn_to_country_caida_d)

files_in_off_nets_folder = os.listdir(results_folder + "candidate_off-nets_good/")

storeData = {}
hg_keywords_l = list()

for file in files_in_off_nets_folder:
    if '.txt' in file:
        hg_keyword = file.split('.')[0]
        hg_keywords_l.append(hg_keyword)

        if hg_keyword not in storeData:
            storeData[hg_keyword] = {
                "EU": set(),
                "AF": set(),
                "SA": set(),
                "AS": set(),
                "NA": set(),
                "OC": set()
            }

        with open(results_folder+ "candidate_off-nets_good/" + file, 'rt') as f:
            for line in f:
                data = json.loads(line)
                asn = str(data['ASN'])
                if asn in asn_to_country_caida_d:
                    cc = asn_to_country_caida_d[asn]
                    #print(cc)
                    if cc in country_to_continent_d:
                        continent_ = country_to_continent_d[cc]
                        storeData[hg_keyword][continent_].add(asn)
newFolder_path = results_folder + "offnets_to_continents/"

if not os.path.exists(newFolder_path):
    os.makedirs(newFolder_path)

for hg_keyword in hg_keywords_l:
    with open(newFolder_path + hg_keyword + ".txt", 'wt') as fw:
        for continent in storeData[hg_keyword]:
            fw.write("{0}: {1} ASes\n".format(continent, len(storeData[hg_keyword][continent])))
        fw.write('# # #\n')
        for continent in storeData[hg_keyword]:
            fw.write("{0}: {1} \n- - - \n".format(continent, ", ".join(["AS" + str(ASN) for ASN in storeData[hg_keyword][continent]])))